# Implementação de Perceptron Multicamadas (MLP) para Classificação de Texto

Este notebook implementa um modelo de **Perceptron Multicamadas (MLP)** para classificar textos em categorias como 'AI' ou 'Human'. Vamos utilizar os dados de entrada fornecidos, aplicar o pré-processamento, treinar o modelo e gerar previsões para um novo conjunto de dados.

## Passos:
1. Carregar e pré-processar os dados.
2. Implementar o MLP (Perceptron Multicamadas).
3. Treinar o modelo e fazer previsões.
4. Salvar as previsões num arquivo CSV.

In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## 1. Carregar e Pré-processar os Dados

Aqui vamos carregar os dados de entrada e de saída, e realizar o pré-processamento, convertendo os rótulos 'AI' e 'Human' em valores binários (1 para 'AI' e 0 para 'Human'). Além disso, vamos transformar os textos em características numéricas utilizando o TF-IDF (Term Frequency-Inverse Document Frequency).

In [2]:
# Carregar os dados
dataset2_inputs = pd.read_csv('data/dataset2_inputs.csv', sep='\t')
test_in = pd.read_csv('data/test_in.csv', sep='\t')
test_out = pd.read_csv('data/teste_out.csv', sep='\t')

# Pré-processamento dos dados
X_train = test_in['Text']
y_train = test_out['Label'].apply(lambda x: 1 if x == 'AI' else 0)  # 'AI' = 1, 'Human' = 0

# Tokenização e vetorização dos dados utilizando o TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vect = vectorizer.fit_transform(X_train).toarray()

## 2. Implementação do Perceptron Multicamadas (MLP)

Agora, vamos implementar a rede neural do tipo **Perceptron Multicamadas (MLP)** com uma camada oculta. Vamos usar a função sigmoide como ativação e implementar o algoritmo de retropropagação (backpropagation) para otimizar os pesos e o viés da rede.

In [3]:
# Funções auxiliares
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def train_mlp(X_train, y_train, X_test, y_test, hidden_units=10, learning_rate=0.01, epochs=10000):
    # Inicialização dos pesos e viés
    input_layer_size = X_train.shape[1]
    output_layer_size = 1
    np.random.seed(42)

    # Pesos e viés para a camada de entrada para camada oculta
    W1 = np.random.randn(input_layer_size, hidden_units)
    b1 = np.zeros((1, hidden_units))

    # Pesos e viés para a camada oculta para camada de saída
    W2 = np.random.randn(hidden_units, output_layer_size)
    b2 = np.zeros((1, output_layer_size))

    # Lista para salvar os erros durante o treinamento
    errors = []

    # Treinamento
    for epoch in range(epochs):
        # Feedforward - camadas ocultas
        Z1 = np.dot(X_train, W1) + b1
        A1 = sigmoid(Z1)

        # Camada de saída
        Z2 = np.dot(A1, W2) + b2
        A2 = sigmoid(Z2)

        # Cálculo do erro
        error = y_train.reshape(-1, 1) - A2
        errors.append(np.mean(np.abs(error)))

        # Backpropagation
        dA2 = error * sigmoid_derivative(A2)
        dW2 = np.dot(A1.T, dA2)
        db2 = np.sum(dA2, axis=0, keepdims=True)

        dA1 = np.dot(dA2, W2.T) * sigmoid_derivative(A1)
        dW1 = np.dot(X_train.T, dA1)
        db1 = np.sum(dA1, axis=0, keepdims=True)

        # Atualização dos pesos e viés
        W1 += learning_rate * dW1
        b1 += learning_rate * db1
        W2 += learning_rate * dW2
        b2 += learning_rate * db2

        # Printar erro a cada 1000 iterações
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}/{epochs}, Error: {np.mean(np.abs(error))}")

    # Predição nos dados de teste
    Z1 = np.dot(X_test, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)

    predictions = (A2 > 0.5).astype(int).flatten()
    return predictions, errors

## 3. Treinar o Modelo e Fazer Previsões

Agora vamos treinar o modelo MLP utilizando os dados de treino e, em seguida, gerar as previsões para o conjunto de dados `dataset2_inputs`.

In [4]:
# Treinar o modelo MLP
predictions, _ = train_mlp(X_train_vect, y_train, X_test, y_test, hidden_units=20, learning_rate=0.01, epochs=10000)

# Verificar o tamanho das previsões e do dataset2_inputs
print(f"Tamanho do dataset2_inputs: {dataset2_inputs.shape[0]}")
print(f"Tamanho das previsões: {len(predictions)}")

# Ajustar as previsões para garantir que a quantidade de previsões seja a mesma que o número de linhas em dataset2_inputs
if len(predictions) != dataset2_inputs.shape[0]:
    # Se o número de previsões não coincidir, ajustar as previsões
    predictions = predictions[:dataset2_inputs.shape[0]]

# Agora podemos adicionar as previsões corretamente ao DataFrame
predictions_label = label_encoder.inverse_transform(predictions)
dataset2_inputs['Predictions'] = predictions_label

## 4. Salvar as Previsões em um Arquivo CSV

Por fim, vamos salvar as previsões geradas no formato CSV, com as colunas 'ID' e 'Predicted_Label', como solicitado.

In [5]:
# Salvar as previsões num arquivo CSV no mesmo formato de entrada
dataset2_inputs[['ID', 'Predictions']].to_csv('data/dataset2_predictions_mlp.csv', index=False)

print("Previsões geradas e salvas em 'data/dataset2_predictions_mlp.csv'")